In [1]:
import pandas as pd
import os
import sys
import inspect
import s1_parallel_dataset as ca

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Create Parallel Data Corpus


In [2]:
# options: 'open-mistral-nemo', 'mistral-large-latest','gpt-4o-mini','gpt-4o'

df_chats = pd.read_csv('00_input_data/between_us_data_postprocessed.csv')
df_chats.shape

(352, 9)

#### GPT-4o

In [3]:
# this call creates a new dataframe with the parallel corpus, also saved as a csv file

# gpt-4o
df_gpt_pchats = ca.create_parallel_corpus(df_chats, 'gpt-4o')
df_gpt_pchats.head()

Creating Parallel Corpus...: 100%|██████████| 352/352 [04:17<00:00,  1.36it/s]


,messageID,timestamp,original,output,model,prompt_tokens,completion_tokens,object,promptID
0,0,2024-06-27T08:18:26.161Z,What sports do you like to do?,"{\n ""rewrittenSentence"": ""Which sports do you...",gpt-4o-2024-05-13,108,17,chat.completion,111
1,1,2024-06-27T08:18:38.085Z,"I like doing some gym, and taking long walks.","{\n ""rewrittenSentence"": ""I enjoy exercisin...",gpt-4o-2024-05-13,111,17,chat.completion,111
2,2,2024-06-27T08:18:51.835Z,"I'm not so good at ball sports, but I still en...","{\n ""rewrittenSentence"": ""I am not good at ...",gpt-4o-2024-05-13,114,23,chat.completion,111
3,3,2024-06-27T08:18:55.828Z,I like team sports like football or basketball,"{\n ""rewrittenSentence"": ""I like team sport...",gpt-4o-2024-05-13,108,19,chat.completion,111
4,4,2024-06-27T08:19:13.433Z,I am horrible at ball sports hahaha,"{\n ""rewrittenSentence"": ""I am not good at ...",gpt-4o-2024-05-13,107,17,chat.completion,111


In [4]:
df_pchats = ca.parse_parallel_data(df_gpt_pchats)
df_pchats.head()

wronglyParsed:  0


,messageID,timestamp,original,output,model,prompt_tokens,completion_tokens,object,promptID,rewritten
0,0,2024-06-27T08:18:26.161Z,What sports do you like to do?,"{\n ""rewrittenSentence"": ""Which sports do you...",gpt-4o-2024-05-13,108,17,chat.completion,111,Which sports do you enjoy participating in?
1,1,2024-06-27T08:18:38.085Z,"I like doing some gym, and taking long walks.","{\n ""rewrittenSentence"": ""I enjoy exercisin...",gpt-4o-2024-05-13,111,17,chat.completion,111,I enjoy exercising and taking long walks.
2,2,2024-06-27T08:18:51.835Z,"I'm not so good at ball sports, but I still en...","{\n ""rewrittenSentence"": ""I am not good at ...",gpt-4o-2024-05-13,114,23,chat.completion,111,"I am not good at ball sports, but I still enjo..."
3,3,2024-06-27T08:18:55.828Z,I like team sports like football or basketball,"{\n ""rewrittenSentence"": ""I like team sport...",gpt-4o-2024-05-13,108,19,chat.completion,111,I like team sports such as football or basketb...
4,4,2024-06-27T08:19:13.433Z,I am horrible at ball sports hahaha,"{\n ""rewrittenSentence"": ""I am not good at ...",gpt-4o-2024-05-13,107,17,chat.completion,111,I am not good at ball sports.


In [5]:
df_gpt_chats = ca.merge_parallel_data(df_chats,df_pchats)
df_gpt_chats.head()

,timestamp,rewritten,log_level,message,content,fromUserId,toUserId,sessionId,word_count,messageID,parallelSentence
0,2024-06-27T08:18:26.161Z,Which sports do you enjoy participating in?,docker,"{""source"":""stdout"",""log"":""{\""content\"":\""What ...",What sports do you like to do?,bf079e28c74b6404,0475b4a5102ad017,BU01,7,1,None
1,2024-06-27T08:18:38.085Z,I enjoy exercising and taking long walks.,docker,"{""container_name"":""/movez-chat-movez-chat-1"",""...","I like doing some gym, and taking long walks.",f256ae32bf85de79,7881bdd5596a3f86,BU03,9,1,None
2,2024-06-27T08:18:51.835Z,"I am not good at ball sports, but I still enjo...",docker,"{""source"":""stdout"",""log"":""{\""content\"":\""I'm n...","I'm not so good at ball sports, but I still en...",0475b4a5102ad017,bf079e28c74b6404,BU02,12,2,None
3,2024-06-27T08:18:55.828Z,I like team sports such as football or basketb...,docker,"{""container_name"":""/movez-chat-movez-chat-1"",""...",I like team sports like football or basketball,7881bdd5596a3f86,f256ae32bf85de79,BU04,8,2,None
4,2024-06-27T08:19:13.433Z,I am not good at ball sports.,docker,"{""container_id"":""04538756c9cc672ad707e82a4b1cf...",I am horrible at ball sports hahaha,bf079e28c74b6404,0475b4a5102ad017,BU01,7,2,None


In [6]:
df_gpt_chats.to_csv('01_processed_input_data/between_us_data_parallel_gpt.csv', index=False)


#### Mistral-large-latest

In [7]:
# mistral-large-latest
df_mistral_pchats = ca.create_parallel_corpus(df_chats, 'mistral-large-latest')
df_mistral_pchats.head()

Creating Parallel Corpus...: 100%|██████████| 352/352 [06:49<00:00,  1.16s/it]


,messageID,timestamp,original,output,model,prompt_tokens,completion_tokens,object,promptID
0,0,2024-06-27T08:18:26.161Z,What sports do you like to do?,"{""rewrittenSentence"": ""Which sports do you enj...",mistral-large-latest,107,17,chat.completion,111
1,1,2024-06-27T08:18:38.085Z,"I like doing some gym, and taking long walks.","{""rewrittenSentence"": ""I enjoy exercising at t...",mistral-large-latest,110,21,chat.completion,111
2,2,2024-06-27T08:18:51.835Z,"I'm not so good at ball sports, but I still en...","{\n ""rewrittenSentence"": ""I am not skil...",mistral-large-latest,115,28,chat.completion,111
3,3,2024-06-27T08:18:55.828Z,I like team sports like football or basketball,"{""rewrittenSentence"": ""I enjoy team sports suc...",mistral-large-latest,107,19,chat.completion,111
4,4,2024-06-27T08:19:13.433Z,I am horrible at ball sports hahaha,"{""rewrittenSentence"": ""I am bad at ball sports.""}",mistral-large-latest,108,16,chat.completion,111


In [8]:
df_pchats = ca.parse_parallel_data(df_mistral_pchats)
df_pchats

wronglyParsed:  0


,messageID,timestamp,original,output,model,prompt_tokens,completion_tokens,object,promptID,rewritten
0,0,2024-06-27T08:18:26.161Z,What sports do you like to do?,"{""rewrittenSentence"": ""Which sports do you enj...",mistral-large-latest,107,17,chat.completion,111,Which sports do you enjoy participating in?
1,1,2024-06-27T08:18:38.085Z,"I like doing some gym, and taking long walks.","{""rewrittenSentence"": ""I enjoy exercising at t...",mistral-large-latest,110,21,chat.completion,111,I enjoy exercising at the gym and taking long ...
2,2,2024-06-27T08:18:51.835Z,"I'm not so good at ball sports, but I still en...","{\n ""rewrittenSentence"": ""I am not skil...",mistral-large-latest,115,28,chat.completion,111,"I am not skilled at ball sports, but I still e..."
3,3,2024-06-27T08:18:55.828Z,I like team sports like football or basketball,"{""rewrittenSentence"": ""I enjoy team sports suc...",mistral-large-latest,107,19,chat.completion,111,I enjoy team sports such as football or basket...
4,4,2024-06-27T08:19:13.433Z,I am horrible at ball sports hahaha,"{""rewrittenSentence"": ""I am bad at ball sports.""}",mistral-large-latest,108,16,chat.completion,111,I am bad at ball sports.
...,...,...,...,...,...,...,...,...,...,...
347,347,2024-06-28T09:54:01.405Z,i also think those are boring,"{""rewrittenSentence"": ""Those are also consider...",mistral-large-latest,105,16,chat.completion,111,Those are also considered boring.
348,348,2024-06-28T09:54:27.010Z,just sweaty people running around,"{\n ""rewrittenSentence"": ""People running ...",mistral-large-latest,105,21,chat.completion,111,People running around and sweating.
349,349,2024-06-28T09:54:35.065Z,Exactly wht sports make you feel strassed and ...,"{""rewrittenSentence"": ""Which sports make you f...",mistral-large-latest,113,18,chat.completion,111,Which sports make you feel stressed and tired?
350,350,2024-06-28T09:54:58.740Z,I think it makes me feel stressed because I'm ...,"{""rewrittenSentence"": ""It makes me feel stress...",mistral-large-latest,114,21,chat.completion,111,It makes me feel stressed because I am bad at it.


In [9]:
df_mistral_chats = ca.merge_parallel_data(df_chats,df_pchats)
df_mistral_chats

,timestamp,rewritten,log_level,message,content,fromUserId,toUserId,sessionId,word_count,messageID,parallelSentence
0,2024-06-27T08:18:26.161Z,Which sports do you enjoy participating in?,docker,"{""source"":""stdout"",""log"":""{\""content\"":\""What ...",What sports do you like to do?,bf079e28c74b6404,0475b4a5102ad017,BU01,7,1,None
1,2024-06-27T08:18:38.085Z,I enjoy exercising at the gym and taking long ...,docker,"{""container_name"":""/movez-chat-movez-chat-1"",""...","I like doing some gym, and taking long walks.",f256ae32bf85de79,7881bdd5596a3f86,BU03,9,1,None
2,2024-06-27T08:18:51.835Z,"I am not skilled at ball sports, but I still e...",docker,"{""source"":""stdout"",""log"":""{\""content\"":\""I'm n...","I'm not so good at ball sports, but I still en...",0475b4a5102ad017,bf079e28c74b6404,BU02,12,2,None
3,2024-06-27T08:18:55.828Z,I enjoy team sports such as football or basket...,docker,"{""container_name"":""/movez-chat-movez-chat-1"",""...",I like team sports like football or basketball,7881bdd5596a3f86,f256ae32bf85de79,BU04,8,2,None
4,2024-06-27T08:19:13.433Z,I am bad at ball sports.,docker,"{""container_id"":""04538756c9cc672ad707e82a4b1cf...",I am horrible at ball sports hahaha,bf079e28c74b6404,0475b4a5102ad017,BU01,7,2,None
...,...,...,...,...,...,...,...,...,...,...,...
347,2024-06-28T09:54:01.405Z,Those are also considered boring.,docker,"{""container_id"":""8895013e2e9d3e00a535250c74749...",i also think those are boring,0a88167b60546b3a,9b658ac45526e7be,BU06,6,187,None
348,2024-06-28T09:54:27.010Z,People running around and sweating.,docker,"{""container_id"":""8895013e2e9d3e00a535250c74749...",just sweaty people running around,0a88167b60546b3a,9b658ac45526e7be,BU06,5,190,None
349,2024-06-28T09:54:35.065Z,Which sports make you feel stressed and tired?,docker,"{""source"":""stdout"",""log"":""{\""content\"":\""Exact...",Exactly wht sports make you feel strassed and ...,4ed7d8e521bef3cd,9b658ac45526e7be,BU07,9,60,None
350,2024-06-28T09:54:58.740Z,It makes me feel stressed because I am bad at it.,docker,"{""log"":""{\""content\"":\""I think it makes me fee...",I think it makes me feel stressed because I'm ...,0a88167b60546b3a,9b658ac45526e7be,BU06,13,193,None


In [10]:
df_mistral_chats.to_csv('01_processed_input_data/between_us_data_parallel_mistral.csv', index=False)


### Create k-shot data per participant

In [19]:
df_gpt_chats = pd.read_csv('01_processed_input_data/between_us_data_parallel_gpt.csv')

df_five_shots = ca.get_k_shots(df_gpt_chats,5,'gpt')
df_ten_shots = ca.get_k_shots(df_gpt_chats,10,'gpt')
df_twenty_shots = ca.get_k_shots(df_gpt_chats,20,'gpt')
df_all_shots = ca.get_all_shots(df_gpt_chats,'gpt')


In [20]:
df_mistral_chats = pd.read_csv('01_processed_input_data/between_us_data_parallel_mistral.csv')

df_five_shots = ca.get_k_shots(df_mistral_chats,5,'mistral')
df_ten_shots = ca.get_k_shots(df_mistral_chats,10,'mistral')
df_twenty_shots = ca.get_k_shots(df_mistral_chats,20,'mistral')
df_all_shots = ca.get_all_shots(df_mistral_chats,'mistral')
